<a href="https://colab.research.google.com/github/TheAmirHK/ColumnGenerationDecomposition/blob/main/ColumnGeneration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mip

In [4]:
from mip import Model, xsum, MINIMIZE, INTEGER, CONTINUOUS
from mip import OptimizationStatus

In [61]:
# Original Problem
#-------------
model = Model(sense=MINIMIZE)

x1 = model.add_var(var_type=CONTINUOUS, lb=0)
x2 = model.add_var(var_type=CONTINUOUS, lb=0)
y1 = model.add_var(var_type=CONTINUOUS, lb=0)
y2 = model.add_var(var_type=CONTINUOUS, lb=0)
y3 = model.add_var(var_type=CONTINUOUS, lb=0)

model.objective = 10*x1 + 22*x2 + y1 + 7*y2 + 9*y3

# S.t.
model += x1 + 6*x2 + y1 - 5*y3 >= 15
model += -2*x2 + 2*y1 + y2 <= 8

model.optimize()

if model.status == OptimizationStatus.OPTIMAL:
    print(f"Optimal Solution Found!")
    print(f"x1 = {x1.x}, x2 = {x2.x}")
    print(f"y1 = {y1.x}, y2 = {y2.x}, y3 = {y3.x}")
    print(f"Optimal Objective Value = {model.objective_value}")
else:
    print("No optimal solution found.")


Optimal Solution Found!
x1 = 0.0, x2 = 1.5714285714285714
y1 = 5.571428571428572, y2 = 0.0, y3 = 0.0
Optimal Objective Value = 40.14285714285714


In [65]:
# Decomposed problem
#-------------------


# Master problem
#---------------
master = Model(sense=MINIMIZE)

x1 = master.add_var(var_type=CONTINUOUS, lb=0, name="x1")
x2 = master.add_var(var_type=CONTINUOUS, lb=0, name="x2")

coeff_constr1 = {x1: 1, x2: 6}
coeff_constr2 = {x2: -2}

master += xsum(coeff_constr1[var] * var for var in coeff_constr1) >= 15, "constr1"
master += xsum(coeff_constr2[var] * var for var in coeff_constr2) <= 8, "constr2"
master.objective = 10 * x1 + 22 * x2

# Subproblems
#-----------
improvement = True
while improvement:
    master.optimize()

    # dual values
    pi1 = master.constr_by_name("constr1").pi
    pi2 = master.constr_by_name("constr2").pi

    rc_y1 = 1 - pi1 - 2*pi2
    rc_y2 = 7 - pi2
    rc_y3 = 9 + 5 * pi1

    min_rc = min(rc_y1, rc_y2, rc_y3)

    if min_rc >= -1e-6:
        improvement = False
    else:
        master.remove(master.constr_by_name("constr1"))
        master.remove(master.constr_by_name("constr2"))

        if min_rc == rc_y1:
            y = master.add_var(var_type=CONTINUOUS, lb=0, name=f"y1_{master.num_cols}")
            coeff_constr1[y] = 1
            coeff_constr2[y] = 2
            master.objective += 1 * y
        elif min_rc == rc_y2:
            y = master.add_var(var_type=CONTINUOUS, lb=0, name=f"y2_{master.num_cols}")
            coeff_constr2[y] = 1
            master.objective += 7 * y
        else:
            y = master.add_var(var_type=CONTINUOUS, lb=0, name=f"y3_{master.num_cols}")
            coeff_constr1[y] = -5
            master.objective += 9 * y

        master += xsum(coeff_constr1[var] * var for var in coeff_constr1) >= 15, "constr1"
        master += xsum(coeff_constr2[var] * var for var in coeff_constr2) <= 8, "constr2"

if master.status == OptimizationStatus.OPTIMAL:
    print("Optimal Solution Found!")
    for v in master.vars:
        print(f"{v.name} = {v.x}")
    print(f"Objective = {master.objective_value}")
else:
    print("No optimal solution found.")

Optimal Solution Found!
x1 = 0.0
x2 = 1.5714285714285714
y1_2 = 5.571428571428572
Objective = 40.14285714285714
